In [ ]:
import requests
import pandas as pd
import numpy as np

def get_param_column(url, param_number):
    # input:
    #    url: local variable, stores api url
    #    param_number: the index of the parameter
    # output:
    #    df_base: dataframe with POWER metrics
    response = requests.get(url)
    response = response.text
    split_response = response.split('\n')[9:]
    columns = split_response[0].split(',')
    data=','.join(split_response[1:])
    data_list = data.split(',')
    data_list = np.array(data_list[:len(data_list)-1], dtype=float)
    data_array = np.reshape(data_list, (int(len(data_list)/6), 6))
    df_base = pd.DataFrame(data=data_array, columns=columns)
    if param_number > 0:
        df_base.drop(columns=['LAT', 'LON', 'YEAR', 'MO', 'DY'], inplace=True)
        # to avoid duplicate columns when we run the get_param_column function in a loop
    return df_base

parameters = ['ALLSKY_SFC_SW_DWN','ALLSKY_SFC_LW_DWN','ALLSKY_SFC_UV_INDEX','WS2M','T2M','TS','T2M_MAX','T2M_MIN','QV2M','PRECTOTCORR','PS','WS10M','WS10M_MAX','WS10M_MIN','WD10M','WS50M','WS50M_MAX','WS50M_MIN','WD50M']
years = [i for i in range(2000, 2023)]

for year in years:
    for i, param in enumerate(parameters):
        API_URL = f'https://power.larc.nasa.gov/api/temporal/daily/regional?latitude-min=32&latitude-max=42&longitude-min=-120&longitude-max=-110&parameters={param}&community=SB&start={year}0101&end={year}1231&format=CSV'
        if i == 0:
            df = df = get_param_column(API_URL, 0)
        else:
            df = pd.concat([df, get_param_column(API_URL, i)], axis=1)

    df.to_csv(f'California_POWER_{year}.csv')